In [ ]:
import requests
from extract import unsharp, thresh, gray, scale, blur, process_text, extract_text
import download
import uuid

from datetime import datetime

In [ ]:
str(uuid.uuid1())

In [ ]:
url = 'http://localhost:5000'
runner_id = 'jupyter'

In [ ]:
r = requests.get(url + '/api/videotasks', params={'runner': runner_id})
r

In [ ]:
len(r.content)

In [ ]:
task = r.json()
task

In [ ]:
start = datetime.now()

In [ ]:
if task['type'] == 'INFO_REQUEST':
    frames, fps = download.info(task['videoId'])
    response = requests.post(url + '/api/videotasks/info', json={
        'taskId': task['id'],
        'runner': runner_id,
        'frames': frames,
        'fps': fps
    })
    print(response)

In [ ]:
if task['type'] == 'GET_TEXT':
    texts = []
    for i in range(task['frame'], task['frame']+task['count']):
        print('Frame', i)
        image = download.get_frame_num(task['videoId'], i)
        text = process_text(extract_text(unsharp(scale(6, gray(image)))))
        if text:
            print(text)
            texts.append({'frame': i, 'text': text})
    response = requests.post(url + '/api/videotasks/texts', json={
        'taskId': task['id'],
        'runner': runner_id,
        'texts': texts
    })
    print(response)

In [ ]:
(datetime.now()-start).total_seconds() / 60